In [1]:
import pandas as pd
import yfinance as yf 
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

#Getting stock data for Enbridge from the yfinance library/API
enb = yf.Ticker('ENB')
enb_hist = enb.history(period='max')

#Getting 60 days worth of test inputs
enb_test = enb_hist.iloc[-60:]
enb_hist.drop(enb_hist.iloc[-60:].index, inplace=True)

#Slicing to get 2000 data points
enb_hist = enb_hist.iloc[-2000:]

#Converting data into an array to start preprocessing
enb_prices = enb_hist['Close'].values

#Scaling the data to accommodate the magnitude of values
sc = MinMaxScaler(feature_range = (0,1))
scaled_enb_prices = sc.fit_transform(enb_prices.reshape(-1, 1))

#Defining the lookback period and creating the matrix of dependent & independent variables
LOOKBACK = 30 #HYPERPARAMETER

x_train = []
y_train = []

for period in range(LOOKBACK, scaled_enb_prices.shape[0]):
    x_train.append(scaled_enb_prices[period-LOOKBACK:period, 0]) #Appending the previous lookback windows for each period (independent variables)
    y_train.append(scaled_enb_prices[period, 0]) #Appending the LOOKBACK + 1st output for each window (target/dependent variable)

x_train = np.array(x_train)
y_train = np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1)) #Reshaping into a 3d-matrix

#Compiling the TF model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1],1)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units = 50, return_sequences = True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units = 50, return_sequences = True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units = 50, return_sequences = False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units = 1)
])

model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(x_train, y_train, epochs = 50, batch_size = 16)

#Preprocessing the training data to be able to feed into our model
test_set = enb_test['Close'].values
all_data = pd.concat((enb_hist['Close'], enb_test['Close']), axis = 0)

inputs = all_data[len(all_data) - len(test_set) - LOOKBACK :].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

x_test = []
for period in range(LOOKBACK, LOOKBACK + len(test_set)): #Size of the data
    x_test.append(inputs[period-LOOKBACK:period, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

#Predicting the model and removing the scale to compare against actuals
pred = model.predict(x_test)
pred = sc.inverse_transform(pred)

#Plotting results
plt.figure(figsize=(8, 6), dpi=150)
plt.plot(test_set, color = 'red', label = 'Actual Stock Price')
plt.plot(pred, color = 'green', label = 'Predicted Stock Price')
plt.title('ENB Price Prediction')
plt.xlabel('Time (in days)')
plt.ylabel('Stock Price')
plt.legend()
plt.show()